<a href="https://colab.research.google.com/github/UmarNauruzov/recommendation_otel/blob/main/recommendation_otel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Знакомство с датасетом

## Получение данных

In [ ]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/TRAIN.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data/')

Возьмем небольшой фрагмент данных для ознакомления с датасетом и построим некоторые графики для того, чтобы погрузиться в предметную область и иметь некоторое представление об имеющихся данных.

In [ ]:
train_df = pd.read_csv('/content/data/train.csv', nrows=20000)
test_df = train_df[10000:]

In [ ]:
train_df.info()
print("----------------------------")
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   date_time                  20000 non-null  object 
 1   site_name                  20000 non-null  int64  
 2   posa_continent             20000 non-null  int64  
 3   user_location_country      20000 non-null  int64  
 4   user_location_region       20000 non-null  int64  
 5   user_location_city         20000 non-null  int64  
 6   orig_destination_distance  12599 non-null  float64
 7   user_id                    20000 non-null  int64  
 8   is_mobile                  20000 non-null  int64  
 9   is_package                 20000 non-null  int64  
 10  channel                    20000 non-null  int64  
 11  srch_ci                    19983 non-null  object 
 12  srch_co                    19983 non-null  object 
 13  srch_adults_cnt            20000 non-null  int

In [ ]:
# предварительный просмотр данных
train_df.head()

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-07-15 11:37:29,2,3,215,611,21634,NaN,177002,0,1,...,0,2,8271,1,0,1,2,50,696,21
1,2013-03-13 17:23:43,2,3,66,392,32518,143.4471,190606,0,0,...,0,1,8267,1,0,3,2,50,675,17
2,2013-04-30 21:59:08,24,2,3,60,4699,NaN,208044,0,0,...,0,1,8255,1,0,1,6,77,20,9
3,2013-11-22 13:48:19,2,3,66,322,29466,908.9419,268453,0,0,...,0,1,8267,1,0,1,2,50,675,55
4,2014-08-10 18:52:36,11,3,205,155,53927,87.6362,204117,0,0,...,0,1,12943,5,0,1,2,198,399,6


Описание полей датасета


|Column name|Description|Data type|
|-----------|-----------|---------|
|date_time|Timestamp|string|
|site_name|ID of the Expedia point of sale (i.e. Expedia.com, Expedia.co.uk, Expedia.co.jp, ...)|int|
|posa_continent|ID of continent associated with site_name|int|
|user_location_country|The ID of the country the customer is located|int|
|user_location_region|The ID of the region the customer is located|int|
|user_location_city|The ID of the city the customer is located|int|
|orig_destination_distance|Physical distance between a hotel and a customer at the time of search. A null means the distance could not be calculated|double|
|user_id|ID of user|int|
|is_mobile|1 when a user connected from a mobile device, 0 otherwise|tinyint|
|is_package|1 if the click/booking was generated as a part of a package (i.e. combined with a flight), 0 otherwise|int|
|channel|ID of a marketing channel|int|
|srch_ci|Checkin date|string|
|srch_co|Checkout date|string|
|srch_adults_cnt|The number of adults specified in the hotel room|int|
|srch_children_cnt|The number of (extra occupancy) children specified in the hotel room|int|
|srch_rm_cnt|The number of hotel rooms specified in the search|int|
|srch_destination_id|ID of the destination where the hotel search was performed|int|
|srch_destination_type_id|Type of destination|int|
|hotel_continent|Hotel continent|int|
|hotel_country|Hotel country|int|
|hotel_market|Hotel market|int|
|is_booking|1 if a booking, 0 if a click|tinyint|
|cnt|Numer of similar events in the context of the same user session|bigint|
|hotel_cluster|ID of a hotel cluster|int|

## Предобработка данных

В качестве первого шага очистим данные, выполним предварительную обработку и проведем разведочный анализ для того, чтобы сформировать представление о процедуре выбора отеля.


* Удалим пользователей, которые не забронировали отель
* Определим запросы пользователей, относящиеся к конкретному типу направления (месту пребывания)
* orig_destination_distance содержит значения NaN
* Рассмотрим даты заезда и выезда, чтобы узнать продолжительность пребывания для каждой записи в тренировочном наборе.


In [ ]:
import warnings
warnings.filterwarnings('ignore')
from matplotlib import rcParams
import pandas as pd

# Plot 
bookings_df = train_df[train_df["is_booking"] == 1]

# figure size in inches
rcParams['figure.figsize'] = 20,10

# Из каких стран чаще всего едут клиенты?
country_counts = bookings_df['user_location_country'].value_counts().reset_index()
sns.countplot(x='index', data=country_counts, order=country_counts['user_location_country'].index)
plt.xticks(rotation=90)


In [ ]:
import warnings
warnings.filterwarnings('ignore')
from matplotlib import rcParams

# Plot 
bookings_df = train_df[train_df["is_booking"] == 1]

# figure size in inches
rcParams['figure.figsize'] = 20,10
# Из каких стран чаще всего едут клиенты?
#sns.countplot(data=bookings_df, x='user_location_country', order=bookings_df['user_location_country'].value_counts().index)
sns.countplot(x=bookings_df['user_location_country'], order = bookings_df['user_location_country'].value_counts().index)
plt.xticks(rotation=90)

In [ ]:
#В какие страны чаще всего ездят клиенты? 
sns.countplot(data=bookings_df, x='hotel_country', order = bookings_df['hotel_country'].value_counts().index)
plt.xticks(rotation=90)

In [ ]:
# Куда едет большинство клиентов из определенной страны?
user_country_id = 66
country_customers = train_df[train_df["user_location_country"] == user_country_id]
sns.countplot(data=country_customers, x='hotel_country', order=country_customers['hotel_country'].value_counts().index)
plt.xticks(rotation=90)

In [ ]:
# График частот для каждого hotel_clusters
sns.countplot(data=train_df, x="hotel_cluster", order = train_df["hotel_cluster"].value_counts().index)
plt.xticks(rotation=90)

In [ ]:
# Отели какого кластера наиболее часто бронируют клиенты из определенной страны?
customer_clusters = train_df[train_df["user_location_country"] == user_country_id]["hotel_cluster"]
sns.countplot(data=train_df[train_df["user_location_country"] == user_country_id], x='hotel_cluster', order=train_df[train_df["user_location_country"] == user_country_id]['hotel_cluster'].value_counts().index)
#sns.countplot(customer_clusters, order = customer_clusters.value_counts().index)
plt.xticks(rotation=90)

In [ ]:
# Какие гостиничные кластеры наиболее часто встречаются в стране?
country_id = 50
country_clusters = train_df[train_df["hotel_country"] == country_id]["hotel_cluster"]
sns.countplot(x= country_clusters, order = country_clusters.value_counts().index)
plt.xticks(rotation=90)

In [ ]:
# Частота для каждого posa_continent
sns.countplot(data=train_df, x='posa_continent', order=[0,1,2,3,4])

In [ ]:
# Частота для каждого posa_continent с разбивкой по hotel_continent
sns.countplot(data=train_df, x= 'posa_continent', order=[0,1,2,3,4], hue=train_df['hotel_continent'])

In [ ]:
# Частота для каждого hotel_continent
sns.countplot(x ='hotel_continent', data=train_df,order=[0,2,3,4,5,6])

In [ ]:
# Частота для каждого hotel_continent с разбивкой по posa_continent
sns.countplot(x='hotel_continent', hue='posa_continent', data=train_df, order=[0,2,3,4,5,6])

In [ ]:
# Частота бронирований с мобильных телефонов
sns.countplot(x='is_mobile',data=bookings_df, order=[0,1])

In [ ]:
# Частота бронирований отелей вместе с другими услугами (is_package)
sns.countplot(x='is_package',data=bookings_df, order=[0,1])

In [ ]:
# Наиболее влиятельный канал
sns.countplot(x='channel', order=list(range(0,10)), data=train_df)

In [ ]:
# heatmap
sns.heatmap(train_df.corr(),cmap='coolwarm', annot=True,linewidths=2)

## Формирование новых признаков

Зачастую бывает удобно сформировать новые признаки на основе уже имеющихся. Эти новые признаки, возможно, будут более ценны с точки зрения использования в предсказании. 

**Дополнительные признаки на основе даты**
* stay_dur: количество дней пребывания
* no_of_days_bet_booking: количество дней между бронированием и
  * Cin_day: Днем Check-in
  * Cin_month: Месяцем Check-in
  * Cin_year: Годом Check-in

In [ ]:
# Function to convert date object into relevant attributes
def convert_date_into_days(df):
    df['srch_ci'] = pd.to_datetime(df['srch_ci'])
    df['srch_co'] = pd.to_datetime(df['srch_co'])
    df['date_time'] = pd.to_datetime(df['date_time'])
    
    df['stay_dur'] = (df['srch_co'] - df['srch_ci']).astype('timedelta64[D]')
    df['no_of_days_bet_booking'] = (df['srch_ci'] - df['date_time']).astype('timedelta64[D]')
    
    # For hotel check-in
    # Month, Year, Day
    df['Cin_day'] = df["srch_ci"].apply(lambda x: x.day)
    df['Cin_month'] = df["srch_ci"].apply(lambda x: x.month)
    df['Cin_year'] = df["srch_ci"].apply(lambda x: x.year)
  

In [ ]:
convert_date_into_days(train_df)
convert_date_into_days(test_df)

In [ ]:
# Количество бронировний по месяцам
sns.countplot(x='Cin_month',data=train_df[train_df["is_booking"] == 1],order=list(range(1,13)))

In [ ]:
# Количество бронировний по дням месяца
sns.countplot(x='Cin_day',data=train_df[train_df["is_booking"] == 1],order=list(range(1,32)))

In [ ]:
# Частоты длительностей пребывания
sns.countplot(x='stay_dur',data=train_df[train_df["is_booking"] == 1])

In [ ]:
# Уберем ненужные признаки, которые не будут полезны для анализа и предсказания
#test_user_id = test_df['user_id']
columns = ['date_time', 'srch_ci','user_id','srch_destination_type_id','srch_destination_id', 'site_name', 'user_location_region', 'user_location_city', 
                              'user_id', 'srch_co', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt']
train_df.drop(columns=columns,axis=1,inplace=True)
test_df.drop(columns=columns,axis=1,inplace=True)

<ipython-input-11-a7920c45ccc1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(columns=columns,axis=1,inplace=True)


In [ ]:
# Определим процент значений NaN по каждому признаку
total = train_df.isnull().sum().sort_values(ascending=False)
percent = (train_df.isnull().sum()/train_df['hotel_cluster'].count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data

,Total,Percent
orig_destination_distance,7401,0.37005
Cin_year,17,0.00085
Cin_month,17,0.00085
Cin_day,17,0.00085
no_of_days_bet_booking,17,0.00085
stay_dur,17,0.00085
hotel_country,0,0.00000
hotel_cluster,0,0.00000
hotel_market,0,0.00000
posa_continent,0,0.00000


In [ ]:
# Заменим NaN датой с наибольшей частотой
train_df['Cin_day'] = train_df['Cin_day'].fillna(26.0)
train_df['Cin_month'] = train_df['Cin_month'].fillna(8.0)
train_df['Cin_year'] = train_df['Cin_year'].fillna(2014.0)
train_df['stay_dur'] = train_df['stay_dur'].fillna(1.0)
train_df['no_of_days_bet_booking'] = train_df['no_of_days_bet_booking'].fillna(0.0)

# Заменим NaN на среднее значение по столбцу
train_df['orig_destination_distance'].fillna(train_df['orig_destination_distance'].mean(), inplace=True)

# Предсказание

Переходим непосредственно к построению рекомендательной системы. Напомним, что основная задача — порекомендовать пользователю отель в месте его назначения.

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

# 1. Загрузка данных
data = pd.read_csv('/content/data/train.csv', nrows=90000)
len(data)

90000

In [ ]:
# Function to convert date object into relevant attributes
def convert_date_into_days(df):
    df['srch_ci'] = pd.to_datetime(df['srch_ci'])
    df['srch_co'] = pd.to_datetime(df['srch_co'])
    df['date_time'] = pd.to_datetime(df['date_time'])
    
    df['stay_dur'] = (df['srch_co'] - df['srch_ci']).astype('timedelta64[D]')
    df['no_of_days_bet_booking'] = (df['srch_ci'] - df['date_time']).astype('timedelta64[D]')
    
    # For hotel check-in
    # Month, Year, Day
    df['Cin_day'] = df["srch_ci"].apply(lambda x: x.day)
    df['Cin_month'] = df["srch_ci"].apply(lambda x: x.month)
    df['Cin_year'] = df["srch_ci"].apply(lambda x: x.year)

In [ ]:
convert_date_into_days(data)

In [ ]:
# Уберем ненужные признаки, которые не будут полезны для анализа и предсказания
#test_user_id = test_df['user_id']
columns = ['date_time', 'srch_ci','user_id','srch_destination_type_id','srch_destination_id', 'site_name', 'user_location_region', 'user_location_city', 
                              'user_id', 'srch_co', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt']
data.drop(columns=columns,axis=1,inplace=True)


In [ ]:
# Определим процент значений NaN по каждому признаку
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data['hotel_cluster'].count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data

,Total,Percent
orig_destination_distance,33236,0.369289
Cin_year,63,0.000700
Cin_month,63,0.000700
Cin_day,63,0.000700
no_of_days_bet_booking,63,0.000700
stay_dur,63,0.000700
hotel_country,0,0.000000
hotel_cluster,0,0.000000
hotel_market,0,0.000000
posa_continent,0,0.000000


In [ ]:
# Заменим NaN датой с наибольшей частотой
data['Cin_day'] = data['Cin_day'].fillna(26.0)
data['Cin_month'] = data['Cin_month'].fillna(8.0)
data['Cin_year'] = data['Cin_year'].fillna(2014.0)
data['stay_dur'] = data['stay_dur'].fillna(1.0)
data['no_of_days_bet_booking'] = data['no_of_days_bet_booking'].fillna(0.0)

# Заменим NaN на среднее значение по столбцу
data['orig_destination_distance'].fillna(data['orig_destination_distance'].mean(), inplace=True)

In [ ]:
# 3. Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(data.drop(['hotel_cluster'], axis=1), data['hotel_cluster'], test_size=0.2)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# # Задаем значения параметров, которые нужно оптимизировать
# param_grid = {
#     'n_estimators': [50, 100, 150],
#     'max_depth': [5, 10, 15],
#     'min_samples_leaf': [1, 2, 4],
# }

# # Создаем модель случайного леса
# clf = RandomForestClassifier(random_state=42)

# # Находим оптимальные параметры с помощью поиска по сетке
# grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5)
# grid_search.fit(X_train, y_train)

# # Выводим наилучшие параметры
# print("Best parameters:", grid_search.best_params_)

# Строим модель с наилучшими параметрами
best_clf = RandomForestClassifier(n_estimators=150,
                                   max_depth=15,
                                   min_samples_leaf=1,
                                   random_state=42)
best_clf.fit(X_train, y_train)

# Оценка качества модели
y_pred = best_clf.predict(X_test)
print("Precision:", precision_score(y_test, y_pred, average='macro'))

Precision: 0.3396235000610647


In [ ]:
from sklearn.metrics import accuracy_score
# Оценка качества модели
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 27.72%


In [ ]:
test_data = pd.read_csv('/content/data/test_files/Soc_Net_Task_5_Test_File_0.csv')




In [ ]:
convert_date_into_days(test_data)

In [ ]:
test_data.drop(columns=columns,axis=1,inplace=True)

In [ ]:
# Заменим NaN датой с наибольшей частотой
test_data['Cin_day'] = test_data['Cin_day'].fillna(26.0)
test_data['Cin_month'] = test_data['Cin_month'].fillna(8.0)
test_data['Cin_year'] = test_data['Cin_year'].fillna(2014.0)
test_data['stay_dur'] = test_data['stay_dur'].fillna(1.0)
test_data['no_of_days_bet_booking'] =test_data['no_of_days_bet_booking'].fillna(0.0)

# Заменим NaN на среднее значение по столбцу
test_data['orig_destination_distance'].fillna(test_data['orig_destination_distance'].mean(), inplace=True)

In [ ]:
test_predictions = best_clf.predict_proba(test_data)
# print(test_predictions)
result = pd.DataFrame(test_predictions, columns=best_clf.classes_) # создание DataFrame с результатами
result = result.apply(lambda x: x.sort_values(ascending=False).head(5).index.tolist(), axis=1) # выбор пяти наиболее вероятных кластеров для каждого запроса
result = result.apply(lambda x: ' '.join(map(str, x))) # преобразование списка в строку
print(result)
result.index.name = 'id' # добавление имени индекса
result.to_csv('/content/result.csv', header=True) # сохранение результата в файл
